## Process Player Scores

### Objective

* Build single file for player scores, merge-compatible with weather model

### Rationale

* Why This?  A nicely formatted set of player scores can be merged into the weather model for predictive analytics.

* Why Me?  I will be building the predictive scoring model, so it makes sense to build all the pipelines

* Why Now?  This step needs to be reproducible before creating the prediction system

### Requirements

* numpy 1.16.4
* pandas 0.24.2


### Input / Output

* Input files are in the `'csv_files` directory of the project root directory (same level as `models`).  The notebook is expected to be in `models/noetbooks`.  Output files will be stored in `models/score_model_data`

* The input files are named via data index as follows: `Chabot_{mm}-{dd}-{yyyy}.csv` 

* The output file will be named `scores.csv`

In [1]:
import numpy as np
import pandas as pd
import re
import glob
import csv
import datetime as dt

In [2]:
def extract_datetime_from_Chabot_filename (filename: str):
    """Given a filename that is expected to refer to a csv file with Lake Chabot Golf Course disc golf scores, 
    return a datetime object with the estimated starting date & time.  Uses hard coded knowledge of Chabot disc golf practices
    for start time.  Returns Jan 1, 1900 for date if date parsing fails."""
    
    # Lake Chabot uses a rolling start from 8AM - 2PM year-round
    start_time = dt.time(8,0,0)
    # Filename format "[parent_dirs][/\]Chabot_mm-dd-yyyy.csv"
    # Remove any parent directories first
    filename = re.split(r'[/\\]',filename)[-1]
    date_portion_of_name = filename.split('.')[0].split('_')[1]
    nums_in_date = date_portion_of_name.split('-')
    try:
        month_num = int(nums_in_date[0])
        day_num = int(nums_in_date[1])
        year_num = int(nums_in_date[2])
    except ValueError:
        month_num = 1
        day_num = 1
        year_num = 1900
    
    extracted_date = dt.date(year_num, month_num, day_num)
    
    return dt.datetime.combine(extracted_date, start_time)

In [3]:
def parse_Chabot_input_to_df (filename : str):
    """Given an input filename that is expected to refer to a csv file with Lake Chabot Golf Course disc golf scores,
    return a Pandas dataframe with starting time, player name, raw score, handicap, and adjusted score"""
    
    try:
        input_df = pd.read_csv(filename, header=1, index_col=None, usecols=['Name','Raw','Handicap','Adjusted'],
                         skip_blank_lines=True)
    except ValueError as e:
        print(f'\nError parsing data from {filename}')
        print(e)
        return None 
    # Remove any lines without a number in 'Raw' by first coercing conversion of non-numbers to NaN
    input_df.Raw = pd.to_numeric(input_df.Raw, errors='coerce')
    input_df = input_df.dropna(subset=['Raw'])
    # Drop any scores of zero or below
    input_df = input_df[input_df['Raw'] > 0]
    if len(input_df) == 0:
        print(f'\nNo useable data found in {filename}')
        return None
    datetime_obj = extract_datetime_from_Chabot_filename(filename)
    input_df['time'] = datetime_obj
    input_df['course_id'] = 1
    return input_df

In [4]:
def extract_datetime_from_Golden_Gate_filename (filename: str):
    """Given a filename that is expected to refer to a csv file with Golden Gate Park disc golf scores, return a datetime
    object with the estimated starting date & time.  Uses hard coded knowledge of Golden Gate disc golf practices
    for start time.  Returns Jan 1, 1900 for date if date parsing fails."""
    
    # Golden Gate uses a rolling start time from 7:30 AM onward 
    start_time = dt.time(7,30,0)
    # Filename format "[parent_dirs][/\][m]m-[d]d-yy.csv"
    # Remove any parent directories first
    filename = re.split(r'[/\\]',filename)[-1]
    date_portion_of_name = filename.split('.')[0]
    nums_in_date = date_portion_of_name.split('-')
    try:
        month_num = int(nums_in_date[0])
        day_num = int(nums_in_date[1])
        year_num = int(nums_in_date[2]) + 2000
    except ValueError:
        month_num = 1
        day_num = 1
        year_num = 1900
    
    extracted_date = dt.date(year_num, month_num, day_num)
    
    return dt.datetime.combine(extracted_date, start_time)

In [5]:
def parse_Golden_Gate_input_to_df (filename : str):
    """Given an input filename that is expected to refer to a csv file with Golden Gate Park disc golf scores,
    return a Pandas dataframe with starting time, player name, raw score, handicap, adjusted score and course_id (0)"""
    
    try:
        input_df = pd.read_csv(filename, index_col=None, usecols=['Full Name','Adjust', 'Score'],
                         skip_blank_lines=True)
    except ValueError as e:
        print(f'\nError parsing data from {filename}')
        print(e)
        return None 
    # Synchronize column names with other course data
    input_df = input_df.rename(columns = {'Full Name':'Name','Score':'Raw','Adjust':'Handicap'})
    # Remove any lines without a number in 'Raw' by first coercing conversion of non-numbers to NaN
    input_df.Raw = pd.to_numeric(input_df.Raw, errors='coerce')
    input_df = input_df.dropna(subset=['Raw'])
    # Also remove any scores of zero or below
    input_df = input_df[input_df['Raw'] > 0]
    if len(input_df) == 0:
        print(f'\nNo useable data found in {filename}')
        return None
    # Compute adjusted score
    # Strip parentheses from Handicap and convert to a '-'
    input_df['Handicap'] = input_df['Handicap'].astype('str')
    input_df['Handicap'] = input_df['Handicap'].str.replace('(','-').str.rstrip(')')
    # Replace anything that isn't numeric with NaN
    input_df['Handicap'] = pd.to_numeric(input_df['Handicap'], errors='coerce')
    # Compute adjusted score
    input_df['Adjusted'] = input_df['Raw'] + input_df['Handicap']
    datetime_obj = extract_datetime_from_Golden_Gate_filename(filename)
    input_df['time'] = datetime_obj
    input_df['course_id'] = 0
    return input_df

In [6]:
def find_row_with_dates_in_AP_file(data):
    """Given a two-dimensional list of data from an Aquatic Park yearly csv file, find which row in the file contains
    the dates in m/d/y format.  This row is the last row in the header and is useful in multiple contexts."""
    #Look in column with index '2'
    row_with_dates = 0
    for ix, row in enumerate(data):
        content = row[2]
        if '/' in content:
            row_with_dates = ix
            break  #Once the dates are found, no need to keep looking
    if (row_with_dates == 0):
        print('Unable to find expected date info in file.')
    return row_with_dates

In [7]:
def parse_info_locs_from_AP_file(data, row_with_dates):
    """Given a two-dimensional list of data from an Aquatic Park yearly csv file, and an integer index for the row that 
    contains the dates,return a dictionary mapping dates to olumn index numbers.  The dictionary will have two levels,
    the outer level will hold dates as keys, the inner level will have three keys for each date: 'raw', 'handicap',
    and 'adjusted' for info about the raw score, handicap, and adjusted score that date, respectively.  The info 
    is in the form of the spreadsheet column (or list index #2) that contains this value. The function expects the 
    info type to be in the row above the date."""
    
    result_dict = {}  #Default to empty dict 
    #Now search through the row with dates to find raw score (labeled 'SCORE'),
    #handicap (labeled 'HANDICAP'), and adjusted score (labeled 'ADJUSTED SCORE') in
    #the row with types
    for ix, cell in enumerate(data[row_with_dates]):
        if ix < 2:
            continue    #Skip first two columns
        date_key = cell
        type_label = data[row_with_dates - 1][ix]
        if type_label in ['SCORE','HANDICAP','ADJUSTED SCORE']:
            if date_key not in result_dict:
                result_dict[date_key] = {}
            result_dict[date_key][type_label] = ix
    
    return result_dict

In [8]:
def parse_AP_game_dates(date_string : str):
    """Given a string of the form [m]m/[d]d/yyyy, return a valid Python date object with the given date."""
    
    date_nums = date_string.split('/')
    try:
        month = int(date_nums[0])
        day = int(date_nums[1])
        year = int(date_nums[2]) + 2000
    except ValueError:
        month, day, year = 1, 1, 1900
        
    return dt.date(year, month, day)

In [9]:
def parse_Aquatic_Park_input_to_df(filename: str):
    """Given an input file with a year (or year-to-date)s worth of score data for Berkeley Aquatic Park disc golf course
    in its standard format, return a DataFrame in the standard name, raw score, handicap, adjusted, course_id (2) format.
    This function differs from the other parsers because the file format is substantially different."""
    with open(filename) as csvfile:
        reader = csv.reader(csvfile)
        file_data = [row for row in reader]
    #Create a dictionary with datestrings as keys, each value being a dictionary with 
    #keys 'raw','handicap', and 'adjusted', with values corresponding to column numbers where 
    #this info can be found on the sheet for each date
    last_header_row = find_row_with_dates_in_AP_file(file_data)
    dates_dict = parse_info_locs_from_AP_file(file_data, last_header_row)
    time_list, name_list, raw_list, handicap_list, adjusted_list = ([] for _ in range(5))
    input_df = pd.DataFrame()
    for row_ix, row in enumerate(file_data):
        if row_ix <= last_header_row:
            continue   #Skip row if in header
        player_name = row[1]
        if (len(player_name) > 0) & (player_name != 'Paid Participants:'):
            for col_ix, cell in enumerate(row):
                if file_data[last_header_row - 1][col_ix] == 'SCORE':
                    try:
                        entry = int(cell)
                    except ValueError:
                        continue   #Skip non-numeric entries
                    date_played = file_data[last_header_row][col_ix]
                    score_data = dates_dict.get(date_played,None)
                    if score_data:  #Should always be true, but parsing could be corrupted
                        try:
                            raw_score = int(row[score_data['SCORE']])
                        except ValueError:
                            continue   #ValueError here would mean corrupted parsing, so skip entirely if true
                        try:
                            handicap = int(row[score_data['HANDICAP']])
                            adjusted_score = int(row[score_data['ADJUSTED SCORE']])
                        except ValueError:
                            handicap = 'not established'
                            adjusted_score = 'not established'
                        #Append to the lists for DataFrame
                        time_list.append(dt.datetime.combine(parse_AP_game_dates(date_played),dt.time.fromisoformat('09:00:00')))
                        name_list.append(player_name)
                        raw_list.append(raw_score)
                        handicap_list.append(handicap)
                        adjusted_list.append(adjusted_score)
                    #End of if (score_data)
                elif file_data[last_header_row - 1][col_ix] == 'AVERAGE':  #this signals the end of score data
                    break #Once we hit the end of score data, no need to examine any more columns
            #End of for loop for columns
            print('.',end='')
        #End of check for non-empty player name
    #End of for loop to add rows
    if len(time_list) > 0:
        input_df = pd.DataFrame({'Name':name_list,'Raw':raw_list,'Handicap':handicap_list,
                               'Adjusted':adjusted_list,'time':time_list})
        input_df['course_id'] = 2
    #Remove raw scores of 0 or less
    input_df = input_df[input_df['Raw'] > 0]
    return input_df

In [21]:
# Build a master DataFrame by combining all data from input files
master_df = pd.DataFrame()
print('Extracting files...',end='')
# Chabot data files
for file_to_read in glob.glob('../../csv_files/Chabot_*.csv'):
    master_df = master_df.append(parse_Chabot_input_to_df(file_to_read), sort=False)
    print('+',end='')
# Golden Gate Park data files 
for file_to_read in glob.glob('../../csv_files/[0-9]*.csv'):
    master_df = master_df.append(parse_Golden_Gate_input_to_df(file_to_read), sort=False)
    print ('o',end='')
# Berkeley Aquatic Park files
for file_to_read in glob.glob('../../csv_files/AP*final_week.csv'):
    master_df = master_df.append(parse_Aquatic_Park_input_to_df(file_to_read), sort=False)
    print('x',end='')
print(f'\n{len(master_df)} rows created.')
master_df.head()

Extracting files...+++++++++++++++++++++++++oooooooooooooooooooooooooooo
No useable data found in ../../csv_files\3-31-19.csv
o
No useable data found in ../../csv_files\8-11-13.csv
o.......................................................................................................................................................................................................................................................................x..............................................................................................................................................................................................................................................................................................x...........................................................................................................................................................................................................................................................................

,Name,Raw,Handicap,Adjusted,time,course_id
1,Mark Shannon,61.0,-12.4,48.6,2019-01-19 08:00:00,1
2,Jacob Kermish-Wells,60.0,-10.72,49.28,2019-01-19 08:00:00,1
3,Luiz Celeste,55.0,-5.23,49.77,2019-01-19 08:00:00,1
4,Tu Tran,59.0,-9.07,49.93,2019-01-19 08:00:00,1
5,Matty Angell,53.0,-1.5,51.5,2019-01-19 08:00:00,1


In [22]:
# Clean up DataFrame
# 1) Drop anything with NaN in the name
master_df = master_df.dropna(subset = ['Name'])
# 1) Get rid of anything including and after a '(' in the name
master_df['Name'] = master_df['Name'].apply(lambda x : x.split('(')[0])
# 2) Coerce all non-numeric handicap and adjusted to 'NaN' 
master_df['Handicap'] = pd.to_numeric(master_df['Handicap'], errors='coerce')
master_df['Adjusted'] = pd.to_numeric(master_df['Adjusted'], errors='coerce')
# 3) Reset index and get rid of old index
master_df = master_df.reset_index()
master_df = master_df.drop(columns = 'index')
master_df.head()

,Name,Raw,Handicap,Adjusted,time,course_id
0,Mark Shannon,61.0,-12.40,48.60,2019-01-19 08:00:00,1
1,Jacob Kermish-Wells,60.0,-10.72,49.28,2019-01-19 08:00:00,1
2,Luiz Celeste,55.0,-5.23,49.77,2019-01-19 08:00:00,1
3,Tu Tran,59.0,-9.07,49.93,2019-01-19 08:00:00,1
4,Matty Angell,53.0,-1.50,51.50,2019-01-19 08:00:00,1


In [24]:
master_df.to_csv('../score_model_data/scores.csv')